### Create and train the network to predict steering angle for car in simulator

#### Read the training data and store images and steering measurements in to np arrays

In [8]:
import csv
import numpy as np
import matplotlib.image as mpimg

def get_image_file_path(line, index, base_path):
    orig_path = line[index]
    file_name = orig_path.split('\\')[-1]
    file_path = base_path + file_name
    return file_path
    
images = []
steerings = []
    
def read_dataset(dataset, images, steerings):
    lines = []
    csvpath = '../P3_Support/Training_Data/' + dataset + '/driving_log.csv'
    csvfile = open(csvpath)
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)

    for line in lines:
        new_base_path = '../P3_Support/Training_Data/' + dataset + '/IMG/'     
        center_image = mpimg.imread(get_image_file_path(line, 0, new_base_path))
#        left_image = mpimg.imread(get_image_file_path(line, 0, new_base_path))
#        right_image = mpimg.imread(get_image_file_path(line, 0, new_base_path))
        center_steer = float(line[3])
#        left_steer = center_steer + 0.1
#        right_steer = float(line[3])
        images.append(center_image)        
        steerings.append(center_steer)        
    return
       
read_dataset('t1', images, steerings)
read_dataset('t1_recovery_left', images, steerings)
read_dataset('t1_recovery_right', images, steerings)
#read_dataset('t1_reverse', images, steerings)

#Data agumentation
aug_images = []
aug_steerings = []
for image, steering in zip(cimages, steerings):
    aug_images.append(image)
    aug_steerings.append(steering)
    image_flipped = np.fliplr(image)
    steer_flipped = -steering
    aug_images.append(image_flipped)
    aug_steerings.append(steer_flipped)  
    
X_train = np.array(aug_images)
y_train = np.array(aug_steerings)

print(X_train.shape)
print(y_train.shape)



(3956, 160, 320, 3)
(3956,)


#### Split the training data for validation, construct a network

In [9]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Conv2D, Dropout, Cropping2D
from keras.layers.pooling import MaxPooling2D

model = Sequential()
#Normalization
model.add(Lambda(lambda x: (x/255.0 - 0.5), input_shape=(160,320,3)))
#Cropping
model.add(Cropping2D(cropping=((60, 10), (0, 0))))
#Convolution->relu->MaxPooling
model.add(Conv2D(32, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#Convolution->relu->MaxPooling
model.add(Conv2D(16, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#Convolution->relu->MaxPooling
model.add(Conv2D(4, 5, 5, activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#Dropout
model.add(Dropout(0.5))
#Flatten
model.add(Flatten())
#Fully connected
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(25))
model.add(Dense(10))
model.add(Dense(5))
model.add(Dropout(0.5))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
model.fit(X_train, y_train, validation_split=0.3, shuffle=True, nb_epoch=6)

model.save('model.h5')



Train on 2769 samples, validate on 1187 samples
Epoch 1/6
2769/2769 [==============================] - 118s - loss: 0.0395 - val_loss: 0.0559
Epoch 2/6
2769/2769 [==============================] - 165s - loss: 0.0287 - val_loss: 0.0556
Epoch 3/6
2769/2769 [==============================] - 285s - loss: 0.0287 - val_loss: 0.0561
Epoch 4/6
2769/2769 [==============================] - 255s - loss: 0.0285 - val_loss: 0.0559
Epoch 5/6
2769/2769 [==============================] - 291s - loss: 0.0285 - val_loss: 0.0565
Epoch 6/6
2769/2769 [==============================] - 317s - loss: 0.0285 - val_loss: 0.0559


In [7]:
model.save('model.h5')